# Vorlesung 10.1
# Dateioperationen

---
 **Hinweis:**
 Diese interaktiven Webseiten beschreiben parallel zu den Vorlesungsfolien das jeweilige Stoffgebiet. Zellen mit C Quelltext können mittels der Tastenkombination \[Shift\] + \[Enter\] kompiliert und ausgeführt werden. Es wird Ihnen empfohlen, Änderungen in diversen Zellen vorzunehmen um ein Gefühl für die Sprache C zu entwickeln.
 
---

## Grundkonzepte

Bisher wurden in Vorlesungen Daten nur mit `scanf` von der Standardeingabe gelesen und mit `printf` auf die Standardausgabe geschrieben. Vor allem bei großen Datenmengen ist es aber wichtig auch aus Dateien zu lesen und in Dateien zu schreiben. In der Sprache C werden sogennante **Datenströme** verwendet um mit Dateien zu interagieren.

### Datenströme (*streams*)

Ein Datenstrom ist das Bindeglied zwischen Programm und Betriebssystem über welches Daten ausgetauscht werden. In vorherigen Beispielen wurden nur Datenströme verwendet, die auf die Kommandozeile schreiben oder lesen:

<img src="images/datastreams.png" width=500>

Der Datenaustausch mit einer Datei funktioniert auch mittels Datenströmen. Der Vorteil von Datenströmen ist, dass der Datenaustausch für das Programm immer gleich funktioniert, egal wo die Daten hingesendet werden. Daher muss der Code nur minimal angepasst werden um zum Beispiel in eine Datei auf der Festplatte zu schreiben anstatt auf den Bildschirm:

<img src="images/datastreams_all.png" width=500>

### Gepufferte Datenströme

Datenströme werden verwendet um Daten mit dem Betriebssystem auszutauschen. Datenströme in C sind gepuffert, was bedeutet, dass Daten immer in einen Zwischenspeicher (Puffer) geschrieben werden und von diesem Puffer wieder gelesen werden, wenn das Programm/Betriebssystem bereit dafür ist. 

<img src="images/buffer.png" width=500>

Ein Aufruf von `printf()` bewirkt also, dass der angegebene Text in den Ausgabepuffer geschrieben wird. Sobald das Betriebssystem bereit ist, kann es diesen Puffer leeren und seienen Inhalt auf die Kommandozeile schreiben. Üblicherweise wird das Leeren aber vom Programm gesteuert.
Der Puffer wird immer geleert wenn:
* `fflush()` aufgerufen wird
* das Zeichen `'\n'` in den Datenstrom geschrieben wird
* der Datenstrom mit `fclose()` geschlossen wird
* das Programm beendet wird

Puffer wurden schon in Vorlesung 3.1 erwähnt, als der *Input Buffer* besprochen wurde. Weil die gesamte Eingabe gespeichert wird, muss beim Einlesen von einzelnen Zeichen `getchar()` ausgeführt werden um das zusätzliche Zeichen `\n` zu entfernen. Das ist allerdings nur bei einzelnen Zeichen (`getchar()` oder `scanf("%c")`) notwendig, da `scanf()` sonst führende Leerzeichen, also auch `\n`, überliest.

In [ ]:
/* Beim Input bleibt das Zeichen '\n' im Puffer und wird daher bei scanf("%c", &operator) gelesen */
long zahl;
char operator;
printf("Operand: ");
scanf("%ld", &zahl);
printf("Operator: ");
scanf("%c", &operator);
printf("Sie haben Operand %ld und Operator %c eingegeben.\n", zahl, operator);

In [ ]:
/* Um das zu Umgehen muss das Zeichen '\n' aus dem Puffer entfernt werden */
long zahl;
char operator;
printf("Operand: ");
scanf("%ld", &zahl);
getchar(); /* getchar liest ein Zeichen und verwirft es */
printf("Operator: ");
scanf("%c", &operator);
printf("Sie haben Operand %ld und Operator %c eingegeben.\n", zahl, operator);

### Öffnen und Schließen von Datenströmen

`stdout`, `stdin`, und `stderr` sind über die gesamte Laufzeit eines Programmes verfügbar. Um aber einen Datenstrom zu einer Datei auf der Festplatte zu bekommen, muss er erst geöffnet werden. Sobald ein Datenstrom auf eine Datei zum Schreiben geöffnet wurde, kann das Betriebsystem andere Schreibzugriffe auf diese Datei verbieten. Daher muss der Datenstrom geschlossen werden um die Datei wieder freizugeben. Zum Öffnen, Schließen, und Leeren von Datenströmen gibt es folgende Funktionen:

| Funktion | Rückgabewert | Beschreibung |
| -------- | ------------ | ------------ |
| `fopen(s, m)` | Zeiger auf den Datenstrom | Öffnet einen Datenstrom zur Datei `s` im Modus `m` |
| `fclose(F)` | `0` wenn der Strom geschlossen wurde, anderes bei Fehler | Schließt den Datenstrom `F` und gibt die Datei wieder frei |
| `fflush(F)` | `0` wenn Daten geschrieben wurden, anderes bei Fehler | Leert den Datenstrom `F` und schreibt den Inhalt in die Datei |

Der Modus `m` in `fopen` ist eine Zeichenkette, die angibt wie mit der Datei interagiert wird. Hier gibt es folgende Möglichkeiten:

| Modus | Beschreibung |
| ----- | ------------ |
| `"r"` | Öffnen zum Lesen (* **r**ead*) der Datei. |
| `"r+"` | Öffnen zum Lesen und Schreiben. Wenn die Datei nicht existiert, wird sie **NICHT** erstellt. |
| `"w"` | Öffnen zum Schreiben (* **w**rite*). Wenn die Datei schon existiert, wird sie überschrieben, sonst erstellt. |
| `"w+"` | Öffnen zum Lesen und Schreiben. Wenn die Datei existiert, wird sie überschrieben, sonst erstellt. |
| `"a"` | Öffnen zum Anhängen (* **a**ppend*) an die Datei. Wenn sie schon existiert, wird ans Ende dazu geschrieben, wenn nicht wird sie erstellt. |
| `"a+"` | Wie `"a"`, nur darf nun auch überall in der Datei gelesen werden |


Im folgenden Beispiel wird die Datei "Beispiel.txt" zum Lesen geöffnet und danach wieder geschlossen.

In [ ]:
/* Öffnen und Schließen von Datenströmen */
/* FILE ist der Datentyp des Datenstromes in C */
FILE *datei;
datei = fopen("Beispiel.txt", "r"); /* "r" bedeutet zum Lesen öffnen */
/* Überprüfen ob Datei geöffnet werden konnte */
if(datei == 0) {
    printf("Fehler beim Oeffnen zum Lesen!\n");
} else {
    fclose(datei); /* fclose() darf nur ausgeführt werden, wenn die Datei geöffnet werden konnte! */
}

**ACHTUNG**: Die Datei "Beispiel.txt" ist genau so wie die Vorlesungsfolien schreibgeschützt und kann daher nicht geöffnet werden! Versuchen Sie den Modus zu "w" zu ändern!

### Lesen und Schreiben von Dateien

Zum Lesen und Schreiben von Dateien gibt es Funktionen, die denen zum Lesen/Schreiben auf die Kommandozeile sehr ähnlich sind:

| Funktion | Rückgabewert | Beschreibung |
| -------- | ------------ | ------------ |
| `fprintf(F, f, ...)` | Anzahl der geschriebenen Zeichen, bei Fehler `0` | Wie `printf`, nur wird in den Strom `F` geschrieben |
| `fscanf(F, f, ...)` | Anzahl der gelesenen Variablen, bei Fehler `EOF` | Wie `scanf`, nur wird aus dem Strom `F` gelesen |
| `fputs(s, F)` | Zahl >= 0, bei Fehler `EOF` | Schreibt eine Zeichenkette `s` in den Strom `F` |
| `fgets(s, n, F)` | Zeiger auf `s`, bei Fehler `0` | Liest eine Zeile aus `F` und schreibt die Zeichenkette `s`, aber maximal `n` Zeichen |
| `fputc(c, F)` | `c` als `int`, bei Fehler `EOF` | Schreibt das Zeichen `c` in den Strom `F` |
| `fgetc(F)` | Das gelesene Zeichen als `int`, bei Fehler `EOF` | Liest ein Zeichen aus dem Strom `F` |

Nun können wir die Datei "Beispiel.txt" lesen und auf die Konsole ausgeben:

In [ ]:
/* Auslesen einer ganzen Textdatei */
#define BUFFERLENGTH 256

/* FILE ist der Datentyp des Datenstromes in C */
FILE *datei;
datei = fopen("Beispiel.txt", "r"); /* "r" bedeutet zum Lesen öffnen */

/* Überprüfen ob Datei geöffnet werden konnte */
if(datei == 0) {
    printf("Fehler beim Oeffnen zum Lesen!\n");
} else {
    /* Alles in Stücken von 256 Zeichen auslesen
        und auf die Konsole ausgeben */
    char buffer[BUFFERLENGTH];
    
    while(fgets(buffer, BUFFERLENGTH, datei)) {
        printf("%s", buffer);
    }
    
    /* fclose() darf nur ausgeführt werden, wenn die Datei geöffnet werden konnte! */
    fclose(datei); 
}

In [ ]:
/* Schreiben einer Datei */
#include "stdio.h"
#include "stdlib.h"

int main() {
    FILE *datei;
    /* Nun öffnen wir eine Datei in ihrem JupyterHub home Ordner. 
        Dort dürfen Sie auch Dateien schreiben.
        Die folgenden zwei Zeilen erstellen den richtigen Pfad. */
    char file[256];
    strcat(strcpy(file, getenv("HOME")), "/meineErsteDatei.txt");
    
    printf("Writing to: %s\n", file);
    datei = fopen(file, "w");
    if(datei == 0) {
        printf("Fehler beim Oeffnen zum Schreiben!\n");
    } else {   
        char name[30];
        printf("Geben Sie ihren Namen ein: ");
        scanf("%s", name);

        /* Jetzt schreiben wir name in die Datei */
        fprintf(datei, "Ihr Name ist \"%s\"", name);

        fclose(datei);
        
        printf("Datei erfolgreich geschrieben!\n");
    }
    
    return 0;
}
/* Sehen Sie nun in ihrem "Home" Ordner (Der erste Ordner beim Öffnen der Website) nach und öffnen die Datei! */

### Text versus Binärdateien

Es gibt zwei verschiedene Arten, Daten in einer Datei zu speichern. Als **Text** oder genau so, wie sie im Speicher stehen (**Binär**). Der Unterschied ist beim Speichern von Zahlen erkennbar:

Speichern der Zahl `100000` als Zeichen ([ASCII](https://www.ascii-code.com/)) oder Binär:    
**Anmerkung**: Das Zeichen `'1'` hat im ASCII Code den Wert `49'

| Datentyp | Speichergröße | Repräsentation von `100000` im Speicher |
| -------- | -------------------------- | ------------- |
| Text     | 6 x `char` = 6 Byte | {`00110001`, `00110000`, `00110000`, `00110000`, `00110000`, `00110000`} |
| Binär (`long`) | 1 x `long` = 4 Byte | `00000000000000011000011010100000` |

Speichern der Zahl `1`:

| Datentyp | Speichergröße | Repräsentation von `1` im Speicher |
| -------- | -------------------------- | ------------- |
| Text     | 1 x `char` = 1 Byte | `00110001` |
| Binär (`long`) | 1 x `long` = 4 Byte | `000000000000000000000000000001` |

In der **binären** Repräsentation ist es also egal, wie groß der tatsächliche Wert der Zahl ist, sie belegt immer nur so viel Speicher wie ihr Datentyp (`long` = 4 Byte).

In einer **Text** Repräsentation wird 1 Byte für jedes Zeichen benötigt mit dem die Zahl dargestellt wird.

### Schreiben von Binärdateien

Wenn Dateien auf diese Weise geschrieben werden, können sie auch nur sinnvoll wieder auf diese Weise gelesen werden. Textdateien können zwar Byte für Byte gelesen und umgewandelt werden, aber auch hier ist es sinnvoller sie als Textdateien zu lesen. Um binäre Dateien zu schreiben, werden folgende Funktionen verwendet:

| Funktion | Rückgabewert | Beschreibung |
| -------- | ------------ | ------------ |
| `fread(b, g, n, F)` | Anzahl der gelesenen Elemente, bei Fehler weniger | Liest `n` Elemente der Größe `g` Byte aus `F` und speichert sie in `b` |
| `fwrite(b, g, n, F)` | Anzahl der geschriebenen Elemente, bei Fehler weniger | Schreibt `n` Elemente der Größe `g` Byte von `b` nach `F` |

Wie Sie in folgenden Beispielen sehen werden, ist das Schreiben und Lesen von Binärdateien deutlich komplexer, da wir genau wissen müssen, welche Datentypen in welcher Reihenfolge geschrieben wurden, um sie richtig zu lesen. Daher ist es üblicherweise einfacher Daten als Text zu schreiben.

In [ ]:
/* Schreiben einer Binärdatei */
#include "stdio.h"
#include "stdlib.h"

int main() {
    FILE *datei;
    /* Nun öffnen wir eine Datei in ihrem JupyterHub home Ordner. 
        Dort dürfen Sie auch Dateien schreiben.
        Die folgenden zwei Zeilen erstellen den richtigen Pfad. */
    char file[256];
    strcat(strcpy(file, getenv("HOME")), "/Fibonacci.bin");
    
    printf("Schreiben auf: %s\n", file);
    datei = fopen(file, "w");
    if(datei == 0) {
        printf("Fehler beim Oeffnen zum Schreiben!\n");
    } else {   
        /* Nun schreiben wir die ersten 10 Fibonacci Zahlen */
        long oldTerm = 0;
        long fibonacci = 1;
        long i = 0;
        for(i = 0; i < 10; ++i) {
            long temp = fibonacci;
            
            /* Schreiben einer long Zahl.
                Wir könnten auch ein Feld aus Fibonacci Zahlen erstellen,
                und dann n auf die Anzahl der Elemente setzen. */
            if(fwrite(&fibonacci, sizeof(long), 1, datei) < 1) {
                fprintf(stderr, "Fibonacci Zahl konnte nicht geschrieben werden!\n");
            }
            
            fibonacci += oldTerm;
            oldTerm = temp;
        }

        fclose(datei);
        
        printf("Datei erfolgreich geschrieben!\n");
    }
    
    return 0;
}

In [ ]:
/* Die Datei Fibonacci.bin kann nun nur mittels fread wieder gelesen werden
    ACHTUNG: Die Zelle über dieser muss zuerst ausgeführt werden,
    damit Fibonacci.bin überhaupt existiert */
#include "stdio.h"
#include "stdlib.h"

int main() {
    FILE *datei;
    /* Nun öffnen wir eine Datei in ihrem JupyterHub home Ordner. 
        Dort dürfen Sie auch Dateien schreiben.
        Die folgenden zwei Zeilen erstellen den richtigen Pfad. */
    char file[256];
    strcat(strcpy(file, getenv("HOME")), "/Fibonacci.bin");
    
    printf("Lesen von: %s\n", file);
    datei = fopen(file, "r"); /* jetzt brauchen wir nur Lesezugriff */
    if(datei == 0) {
        printf("Fehler beim Oeffnen zum Schreiben!\n");
    } else {   
        /* Nun lesen wir alle Fibonacci Zahlen */
        long fibonacci = 0;
        
        printf("Fibonacci Zahlen:\n");
        while(fread(&fibonacci, sizeof(long), 1, datei) > 0) {
            printf("%ld\n", fibonacci);
        }

        fclose(datei);
        
        printf("Datei erfolgreich gelesen!\n");
    }
    
    return 0;
}

### Positionierung in einer Datei

Im obigen Beispiel wurde die gesamte Datei "Fibonacci.bin" ausgegeben. Aber wir könnten auch erst alle Zahlen ab der dritten Zahl lesen. Dazu muss die Position in der Datei "weitergeschoben" werden. Dies ist mit folgenden Funktionen möglich:

| Funktion | Rückgabewert | Beschreibung |
| -------- | ------------ | ------------ |
| `fseek(F, n, p)` | `0`, bei Fehler `-1` | Setzt die Position im Stream `F` auf `n` Bytes relativ zur Position `p` |
| `ftell(F)` | Die Position als `long int`, bei Fehler `-1` | Gibt die aktuelle Position in `F` zurück |

Die Position `p` in `fseek` kann einen von 3 Werten annehmen:

| Name | Beschreibung |
| ---- | ------------ |
| `SEEK_SET` | Anfang der Datei |
| `SEEK_CUR` | Derzeitige Position |
| `SEEK_END` | Ender der Datei. Mit `fseek(F, -3, SEEK_END)` wird also auf das drittletzte Element geschoben |

Im folgenden Beispiel sollen Zahlen erst nach der 5. Fibonacci Zahl ausgegeben werden:

In [ ]:
/* Die Datei Fibonacci.bin kann nun nur mittels fread wieder gelesen werden
    ACHTUNG: Die Zelle über dieser muss zuerst ausgeführt werden,
    damit Fibonacci.bin überhaupt existiert */
#include "stdio.h"
#include "stdlib.h"

int main() {
    FILE *datei;
    /* Nun öffnen wir eine Datei in ihrem JupyterHub home Ordner. 
        Dort dürfen Sie auch Dateien schreiben.
        Die folgenden zwei Zeilen erstellen den richtigen Pfad. */
    char file[256];
    strcat(strcpy(file, getenv("HOME")), "/Fibonacci.bin");
    
    printf("Lese von: %s\n", file);
    datei = fopen(file, "r"); /* jetzt brauchen wir nur Lesezugriff */
    if(datei == 0) {
        printf("Fehler beim Oeffnen zum Schreiben!\n");
    } else {   
        /* Nun lesen wir alle Fibonacci Zahlen */
        long fibonacci = 0;
        
        /* Mit fseek auf die 6. Zahl verschieben */
        fseek(datei, 5 * sizeof(long), SEEK_SET);
        
        printf("Fibonacci Zahlen:\n");
        while(fread(&fibonacci, sizeof(long), 1, datei) > 0) {
            printf("%ld\n", fibonacci);
        }

        fclose(datei);
        
        printf("Datei erfolgreich gelesen!\n");
    }
    
    return 0;
}

### Kopieren von Dateien

Zum Kopieren von Dateien bietet sich die Behandlung als binäre Datei natürlich an, weil der genau Inhalt egal ist, solange er nur korrekt kopiert wird. Das heißt wir können Dateien einfach Byte für Byte kopieren:

In [ ]:
/* Die Datei Fibonacci.bin wird nun nach Fibonacci2.bin kopiert */
#include "stdio.h"
#include "stdlib.h"

#define MAX_PATH_LENGTH 256

int copyFile(char *quelle, char *ziel) {
    FILE *origin;
    FILE *target;
    
    if((origin = fopen(quelle, "r"))) {
        if((target = fopen(ziel, "w"))) {
            char temp;
            while(fread(&temp, 1, 1, origin)) {
                if(fwrite(&temp, 1, 1, target) < 1) {
                    fprintf(stderr, "ERROR: Fehler beim Schreiben!\n");
                    return -1;
                }
            }
        } else {
            fprintf(stderr, "ERROR: Ziel konnte nicht geöffnet werden!\n");
            return -1;
        }
    } else {
        fprintf(stderr, "ERROR: Quelle konnte nicht geöffnet werden!\n");
        return -1;
    }
    
    return 0;
}

int main() {
    char quelle[MAX_PATH_LENGTH];
    char ziel[MAX_PATH_LENGTH];
    strcat(strcpy(quelle, getenv("HOME")), "/Fibonacci.bin");
    strcat(strcpy(ziel, getenv("HOME")), "/Fibonacci2.bin");
    
    if(copyFile(quelle, ziel) == 0) {
        printf("Datei erfolgreich kopiert!");
    } else {
        fprintf(stderr, "ERROR: Datei konnte nicht kopiert werden!\n");
    }
    
    return 0;
}